Zillow Kaggle Competition 

This notebook was developed for a Kaggle competition related to Zillow's estimates of home values.  

In the first phase of the competition (submissions were due Oct 16, 2017), the aim was to predict the residual error in Zillow's home value estimates based on a subset of real estate transactions in California in 2016 and 2017.  Please note that residual errors --not the home values themselves -- were being predicted.  

Only the datasets provided by Zillow could be used in the first round (i.e., no supplemental data could be incorporated).   A second phase, allowing for the use of outside data and aimed at building a better model for predicting actual home values, will take place in 2018.   

Below is a link to the Kaggle competition, with complete details on the competition and the core datasets:

https://www.kaggle.com/c/zillow-prize-1#description

In [1]:
import numpy as np
import pandas as pd
import gc

Prior to developing this notebook, I did exploratory data analysis (EDA), drawing partially on notebooks shared by other Kaggle competitors. This notebook picks up the project up after extensive data prep and adds a few final features and data transformations.

Kaggle/Zillow released the core data in two waves -- the first covering 2016 property features and transactions, and the second providing a 2017 year-to-date update.  

The data prep phase of this project involved combining the 2016 and 2017 "properties" datasets, and then joining that combined file with 2016 and 2017 transaction data. 

First, we combine the 2016 and 2017 properties datasets.

In [2]:
# Read in the new properties dataset 
properties_2017=pd.read_csv("properties_2017.csv", low_memory=False)

In [3]:
properties_2017.head(2)

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN


In [4]:
properties_2017.shape

(2985217, 58)

In [5]:
print(properties_2017.columns)

Index(['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildin

In [6]:
# Based on EDA and preliminary modeling, these are the columns we need at this point 
cols_to_use= [
'parcelid',
'bedroomcnt',
'bathroomcnt',
'roomcnt',
'calculatedfinishedsquarefeet',
'finishedsquarefeet12',
'poolcnt',
'buildingclasstypeid',
'yearbuilt',
'lotsizesquarefeet',
'propertylandusetypeid',
'latitude',
'longitude',
'rawcensustractandblock',
'censustractandblock',
'regionidneighborhood',
'regionidcity',
'regionidcounty',
'regionidzip',     
'landtaxvaluedollarcnt',
'structuretaxvaluedollarcnt',
'taxamount',
'taxvaluedollarcnt',
'taxdelinquencyflag']

In [7]:
properties_2017=properties_2017[cols_to_use]

In [8]:
properties_2017.shape

(2985217, 24)

In [9]:
# Rename the columns where both 2016 and 2017 data will be used (in the merged properties dataset; most of the columns have 
# static data that does not change b/w 2016 and 2017) 
properties_2017.rename(columns={'landtaxvaluedollarcnt': 'landtaxvaluedollarcnt_2017', 'structuretaxvaluedollarcnt' : 
                                'structuretaxvaluedollarcnt_2017', 'taxamount' : 'taxamount_2017', 
                                'taxvaluedollarcnt' : 'taxvaluedollarcnt_2017', 
                                'taxdelinquencyflag' : 'taxdelinquencyflag_2017'}, inplace=True)

In [10]:
properties_2017.columns

Index(['parcelid', 'bedroomcnt', 'bathroomcnt', 'roomcnt',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'poolcnt',
       'buildingclasstypeid', 'yearbuilt', 'lotsizesquarefeet',
       'propertylandusetypeid', 'latitude', 'longitude',
       'rawcensustractandblock', 'censustractandblock', 'regionidneighborhood',
       'regionidcity', 'regionidcounty', 'regionidzip',
       'landtaxvaluedollarcnt_2017', 'structuretaxvaluedollarcnt_2017',
       'taxamount_2017', 'taxvaluedollarcnt_2017', 'taxdelinquencyflag_2017'],
      dtype='object')

In [13]:
# Follow same process with 2016 properties dataset 
properties_2016=pd.read_csv("properties_2016.csv", low_memory=False)

In [14]:
properties_2016.shape

(2985217, 58)

In [15]:
properties_2016=properties_2016[cols_to_use]

In [16]:
properties_2016.rename(columns={'landtaxvaluedollarcnt': 'landtaxvaluedollarcnt_2016', 'structuretaxvaluedollarcnt' : 
                                'structuretaxvaluedollarcnt_2016', 'taxamount' : 'taxamount_2016', 
                                'taxvaluedollarcnt' : 'taxvaluedollarcnt_2016', 
                                'taxdelinquencyflag' : 'taxdelinquencyflag_2016'}, inplace=True)

In [17]:
properties_2016.columns

Index(['parcelid', 'bedroomcnt', 'bathroomcnt', 'roomcnt',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'poolcnt',
       'buildingclasstypeid', 'yearbuilt', 'lotsizesquarefeet',
       'propertylandusetypeid', 'latitude', 'longitude',
       'rawcensustractandblock', 'censustractandblock', 'regionidneighborhood',
       'regionidcity', 'regionidcounty', 'regionidzip',
       'landtaxvaluedollarcnt_2016', 'structuretaxvaluedollarcnt_2016',
       'taxamount_2016', 'taxvaluedollarcnt_2016', 'taxdelinquencyflag_2016'],
      dtype='object')

In [18]:
# Check that all IDs are unique  
unique_ids = pd.concat([properties_2016['parcelid'], properties_2017['parcelid']]).unique()

In [19]:
len(unique_ids)

2985217

In [20]:
del unique_ids; gc.collect()

7

In [21]:
# Combine 2016 and 2017 properties datasets 
properties_combined=pd.merge(properties_2017, properties_2016, how='left', on='parcelid', suffixes=('_2017', '_2016'))

In [22]:
properties_combined.columns

Index(['parcelid', 'bedroomcnt_2017', 'bathroomcnt_2017', 'roomcnt_2017',
       'calculatedfinishedsquarefeet_2017', 'finishedsquarefeet12_2017',
       'poolcnt_2017', 'buildingclasstypeid_2017', 'yearbuilt_2017',
       'lotsizesquarefeet_2017', 'propertylandusetypeid_2017', 'latitude_2017',
       'longitude_2017', 'rawcensustractandblock_2017',
       'censustractandblock_2017', 'regionidneighborhood_2017',
       'regionidcity_2017', 'regionidcounty_2017', 'regionidzip_2017',
       'landtaxvaluedollarcnt_2017', 'structuretaxvaluedollarcnt_2017',
       'taxamount_2017', 'taxvaluedollarcnt_2017', 'taxdelinquencyflag_2017',
       'bedroomcnt_2016', 'bathroomcnt_2016', 'roomcnt_2016',
       'calculatedfinishedsquarefeet_2016', 'finishedsquarefeet12_2016',
       'poolcnt_2016', 'buildingclasstypeid_2016', 'yearbuilt_2016',
       'lotsizesquarefeet_2016', 'propertylandusetypeid_2016', 'latitude_2016',
       'longitude_2016', 'rawcensustractandblock_2016',
       'censustractandb

In [23]:
cols_to_use2= ['parcelid', 'bedroomcnt_2017', 'bathroomcnt_2017', 'roomcnt_2017',
       'calculatedfinishedsquarefeet_2017', 'finishedsquarefeet12_2017',
       'poolcnt_2017', 'buildingclasstypeid_2017', 'yearbuilt_2017',
       'lotsizesquarefeet_2017', 'propertylandusetypeid_2017', 'latitude_2017',
       'longitude_2017', 'rawcensustractandblock_2017',
       'censustractandblock_2017', 'regionidneighborhood_2017',
       'regionidcity_2017', 'regionidcounty_2017', 'regionidzip_2017',
       'landtaxvaluedollarcnt_2017', 'structuretaxvaluedollarcnt_2017',
       'taxamount_2017', 'taxvaluedollarcnt_2017', 'taxdelinquencyflag_2017',
       'landtaxvaluedollarcnt_2016', 'structuretaxvaluedollarcnt_2016',
       'taxamount_2016', 'taxvaluedollarcnt_2016', 'taxdelinquencyflag_2016']

In [24]:
properties_combined=properties_combined[cols_to_use2]

In [25]:
properties_combined.shape

(2985217, 29)

In [26]:
properties_combined.head(2)

,parcelid,bedroomcnt_2017,bathroomcnt_2017,roomcnt_2017,calculatedfinishedsquarefeet_2017,finishedsquarefeet12_2017,poolcnt_2017,buildingclasstypeid_2017,yearbuilt_2017,lotsizesquarefeet_2017,...,landtaxvaluedollarcnt_2017,structuretaxvaluedollarcnt_2017,taxamount_2017,taxvaluedollarcnt_2017,taxdelinquencyflag_2017,landtaxvaluedollarcnt_2016,structuretaxvaluedollarcnt_2016,taxamount_2016,taxvaluedollarcnt_2016,taxdelinquencyflag_2016
0,10754147,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,85768.0,...,9.0,NaN,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN
1,10759547,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,4083.0,...,27516.0,NaN,NaN,27516.0,NaN,27516.0,NaN,NaN,27516.0,NaN


In [27]:
# Back up data 
properties_combined.to_csv('properties_combined_base', index=False)

In [28]:
del properties_2016, properties_2017; gc.collect()

340

Prep combined data 

In [29]:
# Treat NaNs for pools as 0; all others are 1 -- a boolean 
properties_combined.poolcnt_2017.fillna(value=0,inplace=True)

In [30]:
# Treat NaNs for taxdelinquency as 0; all others are 1 -- a boolean 
properties_combined.taxdelinquencyflag_2017.fillna(value=0,inplace=True)
properties_combined.taxdelinquencyflag_2016.fillna(value=0,inplace=True)

In [31]:
# Change "Y" to 1 
properties_combined.taxdelinquencyflag_2017=properties_combined.taxdelinquencyflag_2017.replace(to_replace='Y', value=1)
properties_combined.taxdelinquencyflag_2016=properties_combined.taxdelinquencyflag_2016.replace(to_replace='Y', value=1)

In [32]:
# Replace yearbuilt NaNs with median 
median=properties_combined.yearbuilt_2017.median()
median

1963.0

In [33]:
properties_combined.yearbuilt_2017.fillna(median, inplace=True)

In [34]:
# Drop column b/c of excessive NaNs 
print(properties_combined.buildingclasstypeid_2017.isnull().sum())
properties_combined.drop('buildingclasstypeid_2017', axis=1, inplace=True)

2972486


In [35]:
# Room count has too many 0's - drop 
properties_combined.roomcnt_2017.value_counts()[:3]

0.0     2315644
6.0      178563
7.0      156659
8.0      120448
5.0       99734
9.0       45396
4.0       42956
10.0      10897
3.0        6294
Name: roomcnt_2017, dtype: int64

In [36]:
properties_combined.drop('roomcnt_2017', axis=1, inplace=True)

In [37]:
# Excessive NaNs with 'finished square feet 12'
properties_combined.drop('finishedsquarefeet12_2017', axis=1, inplace=True)

Geo-specific transformations 

In [38]:
# Need to divide longitude/latitude by 1,000,000 to make proper geo points 

In [39]:
properties_combined.latitude_2017=properties_combined.latitude_2017.apply(lambda x : x / 1000000)
properties_combined.longitude_2017=properties_combined.longitude_2017.apply(lambda x : x / 1000000)

In [40]:
properties_combined.latitude_2017[:5]

0    34.144442
1    34.140430
2    33.989359
3    34.148863
4    34.194168
Name: latitude_2017, dtype: float64

Google API has limit of 2500 queries (eg, zip codes) per day;  the number of missing zip codes in properties is 12,714; 
the actual number of zip codes we will need is tied more closely to the actual train/test datasets, which is smaller than 
properties; to get around the limit, I am importing zip codes from an earlier geocode of this smaller dataset and combining
with the properties dataset; I will then fill any remaining NaNs with zip code 999999 in the event that a small number of properties w/o zip codes make their way into final test data (we don't know exactly which properties will be sold in last Q of 
2017)

In [ ]:
# # This code works absent the API limit 
# prop_nozip=properties_combined[["parcelid", "regionidzip_2017", 'latitude_2017', 'longitude_2017']]\
# prop_nozip.regionidzip_2017.isnull().sum()
# from pygeocoder import Geocoder
# g=Geocoder()
# properties_combined['regionidzip_derived']=properties_combined.apply(lambda row: g.reverse_geocode(row['latitude_2017'],
#                                                                      row['longitude_2017']).postal_code, axis=1)

In [41]:
# Import previously engineered file of missing zips from training/test data 
propX=pd.read_csv('prop4A.csv')

In [42]:
propX.drop(['latitude', 'longitude', 'regionidzip'], axis=1, inplace=True)

In [43]:
propX.head(3)

,parcelid,regionidzip2
0,76914147,91436
1,76914347,91340
2,76914547,91355


In [44]:
# Merge these zips back into the main properties file 
properties_combined=properties_combined.merge(propX, on='parcelid', how='left')

In [45]:
properties_combined.head(2)

,parcelid,bedroomcnt_2017,bathroomcnt_2017,calculatedfinishedsquarefeet_2017,poolcnt_2017,yearbuilt_2017,lotsizesquarefeet_2017,propertylandusetypeid_2017,latitude_2017,longitude_2017,...,structuretaxvaluedollarcnt_2017,taxamount_2017,taxvaluedollarcnt_2017,taxdelinquencyflag_2017,landtaxvaluedollarcnt_2016,structuretaxvaluedollarcnt_2016,taxamount_2016,taxvaluedollarcnt_2016,taxdelinquencyflag_2016,regionidzip2
0,10754147,0.0,0.0,NaN,0.0,1963.0,85768.0,269.0,34.144442,-118.654084,...,NaN,NaN,9.0,0,9.0,NaN,NaN,9.0,0,NaN
1,10759547,0.0,0.0,NaN,0.0,1963.0,4083.0,261.0,34.140430,-118.625364,...,NaN,NaN,27516.0,0,27516.0,NaN,NaN,27516.0,0,NaN


In [46]:
properties_combined.regionidzip_2017.isnull().sum()

12714

In [47]:
properties_combined.regionidzip_2017.fillna(properties_combined['regionidzip2'], inplace=True)

In [48]:
properties_combined.regionidzip_2017.isnull().sum()

11868

In [49]:
properties_combined.regionidzip_2017.fillna(99999, inplace=True)

In [50]:
properties_combined.regionidzip_2017.isnull().sum()

0

In [51]:
properties_combined.shape

(2985217, 27)

In [52]:
properties_combined.drop(['regionidzip2'], axis=1, inplace=True)

In [53]:
properties_combined.shape

(2985217, 26)

In [77]:
# # Back up 
# properties_combined.to_csv('properties_combined_base2', index=False)

In [54]:
del propX; gc.collect()

565

More data transformations: bed and bath counts, square feet, tax value/amount

In [55]:
properties_combined.isnull().sum()

parcelid                                   0
bedroomcnt_2017                         2945
bathroomcnt_2017                        2957
calculatedfinishedsquarefeet_2017      45097
poolcnt_2017                               0
yearbuilt_2017                             0
lotsizesquarefeet_2017                272706
propertylandusetypeid_2017              2932
latitude_2017                           2932
longitude_2017                          2932
rawcensustractandblock_2017             2932
censustractandblock_2017               74985
regionidneighborhood_2017            1828476
regionidcity_2017                      62128
regionidcounty_2017                     2932
regionidzip_2017                           0
landtaxvaluedollarcnt_2017             59926
structuretaxvaluedollarcnt_2017        46464
taxamount_2017                         22752
taxvaluedollarcnt_2017                 34266
taxdelinquencyflag_2017                    0
landtaxvaluedollarcnt_2016             67733
structuret

In [ ]:
# Use zip code mean to impute missing tax amounts 
grouped=properties_combined.groupby('regionidzip_2017')['taxamount_2017'].aggregate({'taxamount_mean_2017': np.mean}).reset_index()

In [57]:
grouped[:5]

,regionidzip_2017,taxamount_mean_2017
0,90022.0,8357.3100
1,90024.0,32918.1200
2,90025.0,11261.4500
3,90028.0,41769.2940
4,90031.0,1046.3375


In [58]:
properties_combined=properties_combined.merge(grouped, on='regionidzip_2017', how='left')

In [ ]:
grouped=properties_combined.groupby('regionidzip_2017')['taxamount_2016'].aggregate({'taxamount_mean_2016': np.mean}).reset_index()

In [60]:
grouped[:5]

,regionidzip_2017,taxamount_mean_2016
0,90022.0,8357.3100
1,90024.0,32918.1200
2,90025.0,11261.4500
3,90028.0,41769.2940
4,90031.0,1046.3375


In [61]:
properties_combined=properties_combined.merge(grouped, on='regionidzip_2017', how='left')

In [62]:
properties_combined.head(2)

,parcelid,bedroomcnt_2017,bathroomcnt_2017,calculatedfinishedsquarefeet_2017,poolcnt_2017,yearbuilt_2017,lotsizesquarefeet_2017,propertylandusetypeid_2017,latitude_2017,longitude_2017,...,taxamount_2017,taxvaluedollarcnt_2017,taxdelinquencyflag_2017,landtaxvaluedollarcnt_2016,structuretaxvaluedollarcnt_2016,taxamount_2016,taxvaluedollarcnt_2016,taxdelinquencyflag_2016,taxamount_mean_2017,taxamount_mean_2016
0,10754147,0.0,0.0,NaN,0.0,1963.0,85768.0,269.0,34.144442,-118.654084,...,NaN,9.0,0,9.0,NaN,NaN,9.0,0,12789.835052,12773.728669
1,10759547,0.0,0.0,NaN,0.0,1963.0,4083.0,261.0,34.140430,-118.625364,...,NaN,27516.0,0,27516.0,NaN,NaN,27516.0,0,12789.835052,12773.728669


In [63]:
properties_combined.taxamount_2017.isnull().sum()

22752

In [64]:
properties_combined.taxamount_2017.fillna(properties_combined['taxamount_mean_2017'], inplace=True)

In [65]:
properties_combined.taxamount_2017.isnull().sum()

40

In [66]:
# Use overall mean (not by zip code) for the small number NaNs remaining  
properties_combined.taxamount_2017.fillna(properties_combined.taxamount_2017.mean(), inplace=True)

In [67]:
properties_combined.taxamount_2017.isnull().sum()

0

In [68]:
# Repeat for 2016 
properties_combined.taxamount_2016.isnull().sum()

31250

In [69]:
properties_combined.taxamount_2016.fillna(properties_combined['taxamount_mean_2016'], inplace=True)

In [70]:
properties_combined.taxamount_2016.isnull().sum()

40

In [71]:
properties_combined.taxamount_2016.fillna(properties_combined.taxamount_2016.mean(), inplace=True)
properties_combined.taxamount_2016.isnull().sum()

0

In [ ]:
# Same process for taxvalue dollar count 
grouped=properties_combined.groupby('regionidzip_2017')['taxvaluedollarcnt_2017'].aggregate({'taxvaluedollarcnt_2017_mean': np.mean}).reset_index()
properties_combined=properties_combined.merge(grouped, on='regionidzip_2017', how='left')
properties_combined.taxvaluedollarcnt_2017.fillna(properties_combined['taxvaluedollarcnt_2017_mean'], inplace=True)
properties_combined.taxvaluedollarcnt_2017.fillna(properties_combined.taxvaluedollarcnt_2017.mean(), inplace=True)
properties_combined.taxvaluedollarcnt_2017.isnull().sum()

In [ ]:
grouped=properties_combined.groupby('regionidzip_2017')['taxvaluedollarcnt_2016'].aggregate({'taxvaluedollarcnt_2016_mean': np.mean}).reset_index()
properties_combined=properties_combined.merge(grouped, on='regionidzip_2017', how='left')
properties_combined.taxvaluedollarcnt_2016.fillna(properties_combined['taxvaluedollarcnt_2016_mean'], inplace=True)
properties_combined.taxvaluedollarcnt_2016.fillna(properties_combined.taxvaluedollarcnt_2016.mean(), inplace=True)
properties_combined.taxvaluedollarcnt_2016.isnull().sum()

In [ ]:
# Same imputing method with landtaxvaluedollarcnt and structuretaxvaluedollarcnt  
grouped=properties_combined.groupby('regionidzip_2017')['landtaxvaluedollarcnt_2016'].aggregate({'landtaxvaluedollarcnt_2016_mean': np.mean}).reset_index()
properties_combined=properties_combined.merge(grouped, on='regionidzip_2017', how='left')
properties_combined.landtaxvaluedollarcnt_2016.fillna(properties_combined['landtaxvaluedollarcnt_2016_mean'], inplace=True)
properties_combined.landtaxvaluedollarcnt_2016.fillna(properties_combined.landtaxvaluedollarcnt_2016.mean(), inplace=True)
properties_combined.landtaxvaluedollarcnt_2016.isnull().sum()


In [ ]:
grouped=properties_combined.groupby('regionidzip_2017')['landtaxvaluedollarcnt_2017'].aggregate({'landtaxvaluedollarcnt_2017_mean': np.mean}).reset_index()
properties_combined=properties_combined.merge(grouped, on='regionidzip_2017', how='left')
properties_combined.landtaxvaluedollarcnt_2017.fillna(properties_combined['landtaxvaluedollarcnt_2017_mean'], inplace=True)
properties_combined.landtaxvaluedollarcnt_2017.fillna(properties_combined.landtaxvaluedollarcnt_2017.mean(), inplace=True)
properties_combined.landtaxvaluedollarcnt_2017.isnull().sum()

In [ ]:
grouped=properties_combined.groupby('regionidzip_2017')['structuretaxvaluedollarcnt_2016'].aggregate({'structuretaxvaluedollarcnt_2016_mean': np.mean}).reset_index()
properties_combined=properties_combined.merge(grouped, on='regionidzip_2017', how='left')
properties_combined.structuretaxvaluedollarcnt_2016.fillna(properties_combined['structuretaxvaluedollarcnt_2016_mean'], inplace=True)
properties_combined.structuretaxvaluedollarcnt_2016.fillna(properties_combined.structuretaxvaluedollarcnt_2016.mean(), inplace=True)
properties_combined.structuretaxvaluedollarcnt_2016.isnull().sum()

In [ ]:
grouped=properties_combined.groupby('regionidzip_2017')['structuretaxvaluedollarcnt_2017'].aggregate({'structuretaxvaluedollarcnt_2017_mean': np.mean}).reset_index()
properties_combined=properties_combined.merge(grouped, on='regionidzip_2017', how='left')
properties_combined.structuretaxvaluedollarcnt_2017.fillna(properties_combined['structuretaxvaluedollarcnt_2017_mean'], inplace=True)
properties_combined.structuretaxvaluedollarcnt_2017.fillna(properties_combined.structuretaxvaluedollarcnt_2017.mean(), inplace=True)
properties_combined.structuretaxvaluedollarcnt_2017.isnull().sum()

In [78]:
del grouped; gc.collect()
properties_combined.to_csv('properties_combined_base3A', index=False)

In [79]:
# Back up 
# properties_combined=pd.read_csv('properties_combined_base3')

Tax amount is likely a multiple of calculatedfinishedsquarefeet and lotsizesquarefeet; here we will take the avg of calculatedfinishedsquarefeet/taxamount by zip code, and then multiply that by actual tax amount to come up with an imputed calculatedfinishedsquarefeet estimate. Same for lotsizesquarefeet, which is more problematic due to inconsistencies in reporting this (eg, how to handle coops/condos?)

In [ ]:
grouped=properties_combined.groupby('regionidzip_2017')['calculatedfinishedsquarefeet_2017'].aggregate({'calculatedfinishedsquarefeet_2017_mean': np.mean}).reset_index()

In [81]:
grouped.head()

,regionidzip_2017,calculatedfinishedsquarefeet_2017_mean
0,90022.0,3753.0
1,90024.0,3060.5
2,90025.0,816.0
3,90028.0,2159.0
4,90031.0,1438.5


In [82]:
properties_combined=properties_combined.merge(grouped, on='regionidzip_2017', how='left')

In [83]:
properties_combined.columns

Index(['parcelid', 'bedroomcnt_2017', 'bathroomcnt_2017',
       'calculatedfinishedsquarefeet_2017', 'poolcnt_2017', 'yearbuilt_2017',
       'lotsizesquarefeet_2017', 'propertylandusetypeid_2017', 'latitude_2017',
       'longitude_2017', 'rawcensustractandblock_2017',
       'censustractandblock_2017', 'regionidneighborhood_2017',
       'regionidcity_2017', 'regionidcounty_2017', 'regionidzip_2017',
       'landtaxvaluedollarcnt_2017', 'structuretaxvaluedollarcnt_2017',
       'taxamount_2017', 'taxvaluedollarcnt_2017', 'taxdelinquencyflag_2017',
       'landtaxvaluedollarcnt_2016', 'structuretaxvaluedollarcnt_2016',
       'taxamount_2016', 'taxvaluedollarcnt_2016', 'taxdelinquencyflag_2016',
       'taxamount_mean_2017', 'taxamount_mean_2016',
       'taxvaluedollarcnt_2017_mean', 'taxvaluedollarcnt_2016_mean',
       'landtaxvaluedollarcnt_2016_mean', 'landtaxvaluedollarcnt_2017_mean',
       'structuretaxvaluedollarcnt_2016_mean',
       'structuretaxvaluedollarcnt_2017_mean',


In [84]:
properties_combined['taxpersqft_2017']=properties_combined.apply(lambda row: row['taxamount_mean_2017'] / row['calculatedfinishedsquarefeet_2017_mean'], axis=1)

In [85]:
properties_combined.calculatedfinishedsquarefeet_2017.fillna(properties_combined.taxamount_2017*
                                                             properties_combined.taxpersqft_2017, inplace=True)

In [86]:
properties_combined.calculatedfinishedsquarefeet_2017.fillna(properties_combined.calculatedfinishedsquarefeet_2017.mean(),
                                                             inplace=True)

In [87]:
properties_combined.calculatedfinishedsquarefeet_2017.isnull().sum()

0

In [ ]:
grouped=properties_combined.groupby('regionidzip_2017')['lotsizesquarefeet_2017'].aggregate({'lotsizesquarefeet_2017_mean': np.mean}).reset_index()

In [89]:
properties_combined=properties_combined.merge(grouped, on='regionidzip_2017', how='left')

In [90]:
properties_combined['taxperlotsizesqft_2017']=properties_combined.apply(lambda row: row['taxamount_mean_2017'] / row['lotsizesquarefeet_2017_mean'], axis=1)

In [91]:
properties_combined.lotsizesquarefeet_2017.fillna(properties_combined.taxamount_2017*properties_combined.taxperlotsizesqft_2017, inplace=True)

In [92]:
properties_combined.lotsizesquarefeet_2017.fillna(properties_combined.lotsizesquarefeet_2017.mean(), inplace=True)

In [93]:
properties_combined.isnull().sum()

parcelid                                        0
bedroomcnt_2017                              2945
bathroomcnt_2017                             2957
calculatedfinishedsquarefeet_2017               0
poolcnt_2017                                    0
yearbuilt_2017                                  0
lotsizesquarefeet_2017                          0
propertylandusetypeid_2017                   2932
latitude_2017                                2932
longitude_2017                               2932
rawcensustractandblock_2017                  2932
censustractandblock_2017                    74985
regionidneighborhood_2017                 1828476
regionidcity_2017                           62128
regionidcounty_2017                          2932
regionidzip_2017                                0
landtaxvaluedollarcnt_2017                      0
structuretaxvaluedollarcnt_2017                 0
taxamount_2017                                  0
taxvaluedollarcnt_2017                          0


In [94]:
# Some final imputed data; any other columns with NaNs will not be used in training 
properties_combined.bedroomcnt_2017.fillna(properties_combined.bedroomcnt_2017.mean(), inplace=True)

In [95]:
properties_combined.bathroomcnt_2017.fillna(properties_combined.bathroomcnt_2017.mean(), inplace=True)

In [96]:
properties_combined.propertylandusetypeid_2017.value_counts()

261.0    2152863
266.0     483789
246.0     114415
269.0      61559
263.0      59344
248.0      40731
247.0      39977
31.0        9487
265.0       9421
47.0        4379
260.0       2996
267.0       1808
275.0       1219
264.0        260
270.0         36
279.0          1
Name: propertylandusetypeid_2017, dtype: int64

In [97]:
properties_combined.propertylandusetypeid_2017.fillna(0, inplace=True)

In [98]:
properties_combined.longitude_2017.fillna(properties_combined.longitude_2017.mean(), inplace=True)
properties_combined.latitude_2017.fillna(properties_combined.latitude_2017.mean(), inplace=True)

In [99]:
properties_combined.isnull().sum()

parcelid                                        0
bedroomcnt_2017                                 0
bathroomcnt_2017                                0
calculatedfinishedsquarefeet_2017               0
poolcnt_2017                                    0
yearbuilt_2017                                  0
lotsizesquarefeet_2017                          0
propertylandusetypeid_2017                      0
latitude_2017                                   0
longitude_2017                                  0
rawcensustractandblock_2017                  2932
censustractandblock_2017                    74985
regionidneighborhood_2017                 1828476
regionidcity_2017                           62128
regionidcounty_2017                          2932
regionidzip_2017                                0
landtaxvaluedollarcnt_2017                      0
structuretaxvaluedollarcnt_2017                 0
taxamount_2017                                  0
taxvaluedollarcnt_2017                          0


In [100]:
# # Back up
# properties_combined.to_csv("properties_combined_base4A", index=False)

In [101]:
properties_combined.head()

,parcelid,bedroomcnt_2017,bathroomcnt_2017,calculatedfinishedsquarefeet_2017,poolcnt_2017,yearbuilt_2017,lotsizesquarefeet_2017,propertylandusetypeid_2017,latitude_2017,longitude_2017,...,taxvaluedollarcnt_2017_mean,taxvaluedollarcnt_2016_mean,landtaxvaluedollarcnt_2016_mean,landtaxvaluedollarcnt_2017_mean,structuretaxvaluedollarcnt_2016_mean,structuretaxvaluedollarcnt_2017_mean,calculatedfinishedsquarefeet_2017_mean,taxpersqft_2017,lotsizesquarefeet_2017_mean,taxperlotsizesqft_2017
0,10754147,0.0,0.0,50073.210996,0.0,1963.0,85768.0,269.0,34.144442,-118.654084,...,1.039463e+06,998472.184590,528612.676789,551440.077930,485708.517420,504592.075532,3266.814279,3.915079,60737.068949,0.210577
1,10759547,0.0,0.0,50073.210996,0.0,1963.0,4083.0,261.0,34.140430,-118.625364,...,1.039463e+06,998472.184590,528612.676789,551440.077930,485708.517420,504592.075532,3266.814279,3.915079,60737.068949,0.210577
2,10843547,0.0,0.0,73026.000000,0.0,1959.0,63085.0,47.0,33.989359,-118.394633,...,3.833084e+05,359634.598529,229009.244591,244471.324563,130904.378511,139118.655139,1408.784849,3.410213,82139.117171,0.058489
3,10859147,0.0,0.0,5068.000000,0.0,1948.0,7521.0,47.0,34.148863,-118.437206,...,6.180562e+05,578047.275853,343733.921745,366159.969161,234788.273499,252374.714955,2041.361160,3.689034,16125.141004,0.467013
4,10879947,0.0,0.0,1776.000000,0.0,1947.0,8512.0,31.0,34.194168,-118.385816,...,2.893545e+05,275735.793427,147695.846896,155988.736348,128357.995816,133626.972708,1570.397823,2.285011,18881.895456,0.190043


In [102]:
# Parse out the census tract and block column 

In [103]:
properties_combined.rawcensustractandblock_2017[:5]

0    6.037800e+07
1    6.037800e+07
2    6.037703e+07
3    6.037141e+07
4    6.037123e+07
Name: rawcensustractandblock_2017, dtype: float64

In [104]:
type(properties_combined.rawcensustractandblock_2017[4])

numpy.float64

In [105]:
properties_combined.rawcensustractandblock_2017=properties_combined.rawcensustractandblock_2017.round(3)

In [106]:
test=str(properties_combined.rawcensustractandblock_2017[4])
test

'60371232.052'

In [107]:
len(test)

12

In [108]:
county=test[:4]
county

'6037'

In [109]:
tract=test[4:11]
tract

'1232.05'

In [110]:
block=test[11]
block

'2'

In [111]:
# change type of census tract column to string 
properties_combined.rawcensustractandblock_2017=properties_combined.rawcensustractandblock_2017.astype(str) 

In [112]:
properties_combined.rawcensustractandblock_2017.head()

0    60378002.041
1    60378001.011
2    60377030.012
3    60371412.023
4    60371232.052
Name: rawcensustractandblock_2017, dtype: object

In [113]:
properties_combined.rawcensustractandblock_2017.isnull().sum()

0

In [114]:
properties_combined['census_county']=properties_combined.rawcensustractandblock_2017.apply(lambda x: x[:4])       

In [115]:
properties_combined.census_county[:5]

0    6037
1    6037
2    6037
3    6037
4    6037
Name: census_county, dtype: object

In [116]:
properties_combined['census_tract']=properties_combined.rawcensustractandblock_2017.apply(lambda x: x[4:11])

In [117]:
properties_combined['census_tract'][:5]

0    8002.04
1    8001.01
2    7030.01
3    1412.02
4    1232.05
Name: census_tract, dtype: object

In [118]:
properties_combined['census_block']=properties_combined.rawcensustractandblock_2017.apply(lambda x: x[11:])

In [119]:
properties_combined['census_block'][:10]

0    1
1    1
2    2
3    3
4    2
5    1
6    1
7    3
8    3
9    3
Name: census_block, dtype: object

In [120]:
properties_combined.census_block.value_counts()

1    1130530
2     959628
3     559434
4     234671
5      73214
6      18684
7       4335
        3196
8        841
9        684
Name: census_block, dtype: int64

In [121]:
properties_combined.columns

Index(['parcelid', 'bedroomcnt_2017', 'bathroomcnt_2017',
       'calculatedfinishedsquarefeet_2017', 'poolcnt_2017', 'yearbuilt_2017',
       'lotsizesquarefeet_2017', 'propertylandusetypeid_2017', 'latitude_2017',
       'longitude_2017', 'rawcensustractandblock_2017',
       'censustractandblock_2017', 'regionidneighborhood_2017',
       'regionidcity_2017', 'regionidcounty_2017', 'regionidzip_2017',
       'landtaxvaluedollarcnt_2017', 'structuretaxvaluedollarcnt_2017',
       'taxamount_2017', 'taxvaluedollarcnt_2017', 'taxdelinquencyflag_2017',
       'landtaxvaluedollarcnt_2016', 'structuretaxvaluedollarcnt_2016',
       'taxamount_2016', 'taxvaluedollarcnt_2016', 'taxdelinquencyflag_2016',
       'taxamount_mean_2017', 'taxamount_mean_2016',
       'taxvaluedollarcnt_2017_mean', 'taxvaluedollarcnt_2016_mean',
       'landtaxvaluedollarcnt_2016_mean', 'landtaxvaluedollarcnt_2017_mean',
       'structuretaxvaluedollarcnt_2016_mean',
       'structuretaxvaluedollarcnt_2017_mean',


In [122]:
properties_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2985217 entries, 0 to 2985216
Data columns (total 41 columns):
parcelid                                  int64
bedroomcnt_2017                           float64
bathroomcnt_2017                          float64
calculatedfinishedsquarefeet_2017         float64
poolcnt_2017                              float64
yearbuilt_2017                            float64
lotsizesquarefeet_2017                    float64
propertylandusetypeid_2017                float64
latitude_2017                             float64
longitude_2017                            float64
rawcensustractandblock_2017               object
censustractandblock_2017                  float64
regionidneighborhood_2017                 float64
regionidcity_2017                         float64
regionidcounty_2017                       float64
regionidzip_2017                          float64
landtaxvaluedollarcnt_2017                float64
structuretaxvaluedollarcnt_2017         

In [123]:
type(properties_combined.census_county[4])

str

In [124]:
properties_combined.census_county = pd.to_numeric(properties_combined.census_county, errors='coerce')

In [125]:
type(properties_combined.census_county[4])

numpy.float64

In [126]:
properties_combined.census_county[4]

6037.0

In [127]:
properties_combined.census_tract = pd.to_numeric(properties_combined.census_tract, errors='coerce')
properties_combined.census_block = pd.to_numeric(properties_combined.census_block, errors='coerce')

In [128]:
properties_combined.census_block[4]

2.0

In [129]:
properties_combined.census_county.fillna(0,inplace=True)
properties_combined.census_tract.fillna(0,inplace=True)
properties_combined.census_block.fillna(0,inplace=True)

In [130]:
properties_combined.isnull().sum()

parcelid                                        0
bedroomcnt_2017                                 0
bathroomcnt_2017                                0
calculatedfinishedsquarefeet_2017               0
poolcnt_2017                                    0
yearbuilt_2017                                  0
lotsizesquarefeet_2017                          0
propertylandusetypeid_2017                      0
latitude_2017                                   0
longitude_2017                                  0
rawcensustractandblock_2017                     0
censustractandblock_2017                    74985
regionidneighborhood_2017                 1828476
regionidcity_2017                           62128
regionidcounty_2017                          2932
regionidzip_2017                                0
landtaxvaluedollarcnt_2017                      0
structuretaxvaluedollarcnt_2017                 0
taxamount_2017                                  0
taxvaluedollarcnt_2017                          0


In [131]:
# Reorder columns into more intuitive sequence of rows 
properties_combined.columns

Index(['parcelid', 'bedroomcnt_2017', 'bathroomcnt_2017',
       'calculatedfinishedsquarefeet_2017', 'poolcnt_2017', 'yearbuilt_2017',
       'lotsizesquarefeet_2017', 'propertylandusetypeid_2017', 'latitude_2017',
       'longitude_2017', 'rawcensustractandblock_2017',
       'censustractandblock_2017', 'regionidneighborhood_2017',
       'regionidcity_2017', 'regionidcounty_2017', 'regionidzip_2017',
       'landtaxvaluedollarcnt_2017', 'structuretaxvaluedollarcnt_2017',
       'taxamount_2017', 'taxvaluedollarcnt_2017', 'taxdelinquencyflag_2017',
       'landtaxvaluedollarcnt_2016', 'structuretaxvaluedollarcnt_2016',
       'taxamount_2016', 'taxvaluedollarcnt_2016', 'taxdelinquencyflag_2016',
       'taxamount_mean_2017', 'taxamount_mean_2016',
       'taxvaluedollarcnt_2017_mean', 'taxvaluedollarcnt_2016_mean',
       'landtaxvaluedollarcnt_2016_mean', 'landtaxvaluedollarcnt_2017_mean',
       'structuretaxvaluedollarcnt_2016_mean',
       'structuretaxvaluedollarcnt_2017_mean',


In [132]:
properties_combined=properties_combined[['parcelid', 
                                         'calculatedfinishedsquarefeet_2017',
                                         'bedroomcnt_2017', 
                                         'bathroomcnt_2017',
                                         'poolcnt_2017', 
                                         'lotsizesquarefeet_2017',
                                         'yearbuilt_2017',
                                         'propertylandusetypeid_2017',                                      
                                         'latitude_2017',
                                         'longitude_2017', 
                                         'regionidzip_2017', 
                                         'census_county', 
                                         'census_tract', 
                                         'census_block',
                                         # 2016 values 
                                         'landtaxvaluedollarcnt_2016', 
                                         'structuretaxvaluedollarcnt_2016',
                                         'taxvaluedollarcnt_2016', 
                                         'taxdelinquencyflag_2016',
                                         'taxamount_2016',                                                                
                                         # 2017 values 
                                         'landtaxvaluedollarcnt_2017', 
                                         'structuretaxvaluedollarcnt_2017',                                        
                                         'taxvaluedollarcnt_2017', 
                                         'taxdelinquencyflag_2017',
                                         'taxamount_2017',                                                                  
                                         # Remove when training 
                                         'rawcensustractandblock_2017',
                                         'censustractandblock_2017', 
                                         'regionidneighborhood_2017',
                                         'regionidcity_2017', 
                                         'regionidcounty_2017',
                                         'taxamount_mean_2017', 
                                         'taxamount_mean_2016',
                                         'taxvaluedollarcnt_2017_mean',
                                         'taxvaluedollarcnt_2016_mean',
                                         'landtaxvaluedollarcnt_2016_mean',
                                         'landtaxvaluedollarcnt_2017_mean',
                                         'structuretaxvaluedollarcnt_2016_mean',
                                         'structuretaxvaluedollarcnt_2017_mean',
                                         'calculatedfinishedsquarefeet_2017_mean',
                                         'taxpersqft_2017',
                                         'lotsizesquarefeet_2017_mean', 
                                         'taxperlotsizesqft_2017'                        
                                         ]] 

In [133]:
properties_combined.shape

(2985217, 41)

In [34]:
# Back up/read in as needed  
# properties_combined.to_csv("properties_combined_base5A", index=False)
# properties_combined=pd.read_csv("properties_combined_base5A")

In [29]:
properties_combined.dtypes

parcelid                                    int64
calculatedfinishedsquarefeet_2017         float64
bedroomcnt_2017                           float64
bathroomcnt_2017                          float64
poolcnt_2017                              float64
lotsizesquarefeet_2017                    float64
yearbuilt_2017                            float64
propertylandusetypeid_2017                float64
latitude_2017                             float64
longitude_2017                            float64
regionidzip_2017                          float64
census_county                             float64
census_tract                              float64
census_block                              float64
landtaxvaluedollarcnt_2016                float64
structuretaxvaluedollarcnt_2016           float64
taxvaluedollarcnt_2016                    float64
taxdelinquencyflag_2016                     int64
taxamount_2016                            float64
landtaxvaluedollarcnt_2017                float64


In [41]:
# Change dtypes of categorical features to int; needed for use in model 
cat_feat=  ['poolcnt_2017' , 'propertylandusetypeid_2017', 'taxdelinquencyflag_2017', 'taxdelinquencyflag_2016', 
            'regionidzip_2017', 'census_tract', 'census_county', 'census_block']

In [35]:
# Need to change census_tract back to string to remove the decimal 
properties_combined['census_tract']=properties_combined['census_tract'].astype(str)

In [36]:
properties_combined['census_tract'][:5]

0    8002.04
1    8001.01
2    7030.01
3    1412.02
4    1232.05
Name: census_tract, dtype: object

In [39]:
properties_combined['census_tract']=properties_combined['census_tract'].str.replace(".",'')

In [40]:
properties_combined['census_tract'][:5]

0    800204
1    800101
2    703001
3    141202
4    123205
Name: census_tract, dtype: object

In [42]:
properties_combined[cat_feat]=properties_combined[cat_feat].astype(int)

In [43]:
properties_combined.dtypes

parcelid                                    int64
calculatedfinishedsquarefeet_2017         float64
bedroomcnt_2017                           float64
bathroomcnt_2017                          float64
poolcnt_2017                                int32
lotsizesquarefeet_2017                    float64
yearbuilt_2017                            float64
propertylandusetypeid_2017                  int32
latitude_2017                             float64
longitude_2017                            float64
regionidzip_2017                            int32
census_county                               int32
census_tract                                int32
census_block                                int32
landtaxvaluedollarcnt_2016                float64
structuretaxvaluedollarcnt_2016           float64
taxvaluedollarcnt_2016                    float64
taxdelinquencyflag_2016                     int32
taxamount_2016                            float64
landtaxvaluedollarcnt_2017                float64


Now all columns with categorical features can be treated as such in the LGB model 

In [262]:
# # Back up/read in as needed 
# properties_combined.to_csv("properties_combined_base5A", index=False)
# properties_combined=pd.read_csv("properties_combined_base5A")

We now have the core data; where we still have NaNs, the columns were for calc purposes only or will be ignored.

Training datasets....
These contain transaction data -- the logerror from Zillow's predictions vs. actuals and transaction dates.
Merge the original training dataset with the Oct 2 update, and then parse the date for additional features. 

In [263]:
train_2016=pd.read_csv("train_2016_v2.csv", dtype={'parcelid': np.int32, 'logerror' : np.float32, 'transactiondate': object},       
                     parse_dates=['transactiondate'])

In [264]:
train_2016.head(2)

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01


In [265]:
train_2016.shape

(90275, 3)

In [266]:
train_2017=pd.read_csv("train_2017.csv", dtype={'parcelid': np.int32, 'logerror' : np.float32, 'transactiondate': object},       
                     parse_dates=['transactiondate'])

In [267]:
train_2017.head(2)

,parcelid,logerror,transactiondate
0,14297519,0.025595,2017-01-01
1,17052889,0.055619,2017-01-01


In [268]:
train_2017.shape

(77613, 3)

In [269]:
train_combined=train_2016.append(train_2017, ignore_index=True)

In [270]:
train_combined.head(2)

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01


In [271]:
train_combined.shape

(167888, 3)

In [272]:
train_combined['trans_month']=train_combined['transactiondate'].dt.month

In [273]:
train_combined['trans_year']=train_combined['transactiondate'].dt.year

In [274]:
train_combined.trans_year.replace(to_replace=[2016,2017], value=[0, 1], inplace=True)

In [275]:
equiv = {1:1, 2:1, 3:1, 4:2, 5:2, 6:2, 7:3, 8:3, 9:3, 10:4, 11:4, 12:4}
train_combined['trans_qtr']= train_combined['trans_month'].map(equiv)

In [276]:
train_combined.head(2)

,parcelid,logerror,transactiondate,trans_month,trans_year,trans_qtr
0,11016594,0.0276,2016-01-01,1,0,1
1,14366692,-0.1684,2016-01-01,1,0,1


In [277]:
train_combined.shape

(167888, 6)

In [278]:
train_combined.trans_year.value_counts()

0    90275
1    77613
Name: trans_year, dtype: int64

Merge the training data(transctions) with the properties data (core features) to build the final training dataset 

In [279]:
df_train = train_combined.merge(properties_combined, how='left', on='parcelid') 

In [280]:
df_train.shape

(167888, 46)

In [281]:
df_train.head(2)

,parcelid,logerror,transactiondate,trans_month,trans_year,trans_qtr,calculatedfinishedsquarefeet_2017,bedroomcnt_2017,bathroomcnt_2017,poolcnt_2017,...,taxvaluedollarcnt_2017_mean,taxvaluedollarcnt_2016_mean,landtaxvaluedollarcnt_2016_mean,landtaxvaluedollarcnt_2017_mean,structuretaxvaluedollarcnt_2016_mean,structuretaxvaluedollarcnt_2017_mean,calculatedfinishedsquarefeet_2017_mean,taxpersqft_2017,lotsizesquarefeet_2017_mean,taxperlotsizesqft_2017
0,11016594,0.0276,2016-01-01,1,0,1,1684.0,3.0,2.0,0,...,374814.534783,359583.270368,186401.330230,197131.115761,173229.022837,177731.715684,1932.308435,2.425492,20927.554764,0.223953
1,14366692,-0.1684,2016-01-01,1,0,1,2263.0,4.0,3.5,0,...,362939.593701,341396.562758,222749.951286,239483.506153,132784.392839,138008.663032,1750.986164,2.198684,6326.747053,0.608506


In [282]:
df_train.isnull().sum()

parcelid                                       0
logerror                                       0
transactiondate                                0
trans_month                                    0
trans_year                                     0
trans_qtr                                      0
calculatedfinishedsquarefeet_2017              0
bedroomcnt_2017                                0
bathroomcnt_2017                               0
poolcnt_2017                                   0
lotsizesquarefeet_2017                         0
yearbuilt_2017                                 0
propertylandusetypeid_2017                     0
latitude_2017                                  0
longitude_2017                                 0
regionidzip_2017                               0
census_county                                  0
census_tract                                   0
census_block                                   0
landtaxvaluedollarcnt_2016                     0
structuretaxvaluedol

In [283]:
# We need to keep 2016 tax info with 2016 transactions, per Zillow guidance; same for 2017
df_train_2016= df_train.loc[df_train.trans_year==0]

In [284]:
df_train_2016.shape

(90275, 46)

In [285]:
df_train_2017= df_train.loc[df_train.trans_year==1]
df_train_2017.shape

(77613, 46)

In [286]:
df_train_2017.columns

Index(['parcelid', 'logerror', 'transactiondate', 'trans_month', 'trans_year',
       'trans_qtr', 'calculatedfinishedsquarefeet_2017', 'bedroomcnt_2017',
       'bathroomcnt_2017', 'poolcnt_2017', 'lotsizesquarefeet_2017',
       'yearbuilt_2017', 'propertylandusetypeid_2017', 'latitude_2017',
       'longitude_2017', 'regionidzip_2017', 'census_county', 'census_tract',
       'census_block', 'landtaxvaluedollarcnt_2016',
       'structuretaxvaluedollarcnt_2016', 'taxvaluedollarcnt_2016',
       'taxdelinquencyflag_2016', 'taxamount_2016',
       'landtaxvaluedollarcnt_2017', 'structuretaxvaluedollarcnt_2017',
       'taxvaluedollarcnt_2017', 'taxdelinquencyflag_2017', 'taxamount_2017',
       'rawcensustractandblock_2017', 'censustractandblock_2017',
       'regionidneighborhood_2017', 'regionidcity_2017', 'regionidcounty_2017',
       'taxamount_mean_2017', 'taxamount_mean_2016',
       'taxvaluedollarcnt_2017_mean', 'taxvaluedollarcnt_2016_mean',
       'landtaxvaluedollarcnt_2016_

In [287]:
dropcols_2016=[                                                                             
                                         # 2017 values 
                                         'landtaxvaluedollarcnt_2017', 
                                         'structuretaxvaluedollarcnt_2017',                                        
                                         'taxvaluedollarcnt_2017', 
                                         'taxdelinquencyflag_2017',
                                         'taxamount_2017',                                                                  
                                         # Remove when training 
                                         'rawcensustractandblock_2017',
                                         'censustractandblock_2017', 
                                         'regionidneighborhood_2017',
                                         'regionidcity_2017', 
                                         'regionidcounty_2017',
                                         'taxamount_mean_2017', 
                                         'taxamount_mean_2016',
                                         'taxvaluedollarcnt_2017_mean',
                                         'taxvaluedollarcnt_2016_mean',
                                         'landtaxvaluedollarcnt_2016_mean',
                                         'landtaxvaluedollarcnt_2017_mean',
                                         'structuretaxvaluedollarcnt_2016_mean',
                                         'structuretaxvaluedollarcnt_2017_mean',
                                         'calculatedfinishedsquarefeet_2017_mean',
                                         'taxpersqft_2017',
                                         'lotsizesquarefeet_2017_mean', 
                                         'taxperlotsizesqft_2017' ]                      
                                         


dropcols_2017=[
                                         # 2016 values 
                                         'landtaxvaluedollarcnt_2016', 
                                         'structuretaxvaluedollarcnt_2016',
                                         'taxvaluedollarcnt_2016', 
                                         'taxdelinquencyflag_2016',
                                         'taxamount_2016',                                                                
                                                                                                
                                         # Remove when training 
                                         'rawcensustractandblock_2017',
                                         'censustractandblock_2017', 
                                         'regionidneighborhood_2017',
                                         'regionidcity_2017', 
                                         'regionidcounty_2017',
                                         'taxamount_mean_2017', 
                                         'taxamount_mean_2016',
                                         'taxvaluedollarcnt_2017_mean',
                                         'taxvaluedollarcnt_2016_mean',
                                         'landtaxvaluedollarcnt_2016_mean',
                                         'landtaxvaluedollarcnt_2017_mean',
                                         'structuretaxvaluedollarcnt_2016_mean',
                                         'structuretaxvaluedollarcnt_2017_mean',
                                         'calculatedfinishedsquarefeet_2017_mean',
                                         'taxpersqft_2017',
                                         'lotsizesquarefeet_2017_mean', 
                                         'taxperlotsizesqft_2017' ]      





In [288]:
df_train_2016=df_train_2016.drop(dropcols_2016, axis=1)
df_train_2016.shape

(90275, 24)

In [289]:
df_train_2017=df_train_2017.drop(dropcols_2016, axis=1)
df_train_2017.shape

(77613, 24)

In [290]:
# Leave out these addl columns from the training set...(Note: only using trans_quarter, not month)
dropcols_final=['parcelid', 'logerror', 'transactiondate', 'trans_month', 'trans_year']

Modeling....
In previous notebooks, explored use of Random Forests and experimented with parameter tuning; based on results, 
focusing on the Light GBM model here 

In [291]:
import lightgbm as lgb

In [292]:
X_6 = df_train_2016.drop(dropcols_final, axis=1).values
y_6 = df_train_2016['logerror'].values
train_columns_6=df_train_2016.drop(dropcols_final, axis=1).columns
print(X_6.shape, y_6.shape)

(90275, 19) (90275,)


In [293]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_6, y_6, test_size=0.20)

In [294]:
cat_feat_6=['poolcnt_2017',
 'propertylandusetypeid_2017',
 'taxdelinquencyflag_2016',
 'regionidzip_2017',
 'census_tract',
 'census_county',
 'census_block']
cat_feat_7=['poolcnt_2017',
 'propertylandusetypeid_2017',
 'taxdelinquencyflag_2017',
 'regionidzip_2017',
 'census_tract',
 'census_county',
 'census_block']

In [295]:
lgb_train = lgb.Dataset(X_train, label=y_train, feature_name=train_columns_6, categorical_feature=cat_feat_6, free_raw_data=False )
lgb_eval = lgb.Dataset(X_test, label=y_test, feature_name=train_columns_6, categorical_feature=cat_feat_6,free_raw_data=False )

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn import cross_validation

In [ ]:
params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'regression_l1', 
          'nthread': 5, 
          'n_estimators': 20, 
          'num_leaves': 64, 
          'min_data_in_leaf': 100, 
          'learning_rate': 0.002, 
          'metric' : 'mae',
          'max_bin': 512, 
          'subsample_for_bin': 200,
          'subsample': 1, 
          'subsample_freq': 1, 
          'colsample_bytree': 0.8, 
          'reg_alpha': 5, 
          'reg_lambda': 10,
          'min_split_gain': 0.5, 
          'min_child_weight': 1, 
          'min_child_samples': 5, 
          'scale_pos_weight': 1,
          'num_class' : 1,
          'sub_feature': 0.5,
          'num_leaves': 60,
          'min_data': 500,
          'min_hessian': 1
          }


# Create parameters to search
gridParams = {
    'learning_rate': [0.05, 0.1, 0.16],
    'n_estimators': [8,24,48],
    'num_leaves': [6,12,18],
    'min_data_in_leaf': [50, 100,500],
    'max_depth' : [5, 10, 20],
    'max_bin' : [300, 500, 750], 
    'boosting_type' : ['gbdt'],
    'objective' : ['regression_l1'],
    'seed' : [500],
    'colsample_bytree' : [0.65, 0.75, 0.8],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,2],
    'reg_lambda' : [1,2],
    }

# Create classifier to use. 
mdl = lgb.LGBMRegressor(boosting_type= 'gbdt', 
          objective = 'regression_l1', 
          nthread = 5, 
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          subsample_freq = params['subsample_freq'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'], 
          scale_pos_weight = params['scale_pos_weight'])

# To view the default model params:
mdl.get_params().keys()

# Create the grid
grid = GridSearchCV(mdl, gridParams, verbose=1, cv=4, n_jobs=-1)
# Run the grid
grid.fit(X_train, y_train)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)
grid.grid_scores_

In [305]:
best_lgb_6=grid.best_estimator_

In [306]:
best_lgb_6

LGBMRegressor(boosting_type='gbdt', colsample_bytree=0.8, learning_rate=0.16,
       max_bin=750, max_depth=5, min_child_samples=5, min_child_weight=1,
       min_data_in_leaf=50, min_split_gain=0.5, n_estimators=48, nthread=5,
       num_leaves=18, objective='regression_l1', reg_alpha=2, reg_lambda=2,
       scale_pos_weight=1, seed=500, silent=True, subsample=0.7,
       subsample_for_bin=200, subsample_freq=1)

In [259]:
# # After further manual testing of the model 
# best_lgb_6=lgb.LGBMRegressor(boosting_type='gbdt', colsample_bytree=0.8, learning_rate=0.16,
#        max_bin=750, max_depth=10, min_child_samples=5, min_child_weight=1,
#        min_data_in_leaf=50, min_split_gain=0.5, n_estimators=48, nthread=5,
#        num_leaves=12, objective='regression_l1', reg_alpha=1, reg_lambda=1,
#        scale_pos_weight=1, seed=500, silent=True, subsample=0.75,
#        subsample_for_bin=200, subsample_freq=1)

In [307]:
best_lgb_6.fit(X_train, y_train)
print('Start predicting...')
# predict
y_pred = best_lgb_6.predict(X_test, num_iteration=best_lgb_6.best_iteration)


Start predicting...


In [308]:
from sklearn.metrics import mean_absolute_error
print('The mean absolute error of prediction is:', mean_absolute_error(y_test, y_pred) )

The mean absolute error of prediction is: 0.0687120775343


In [309]:
from operator import itemgetter
sorted(list(zip(train_columns_6,best_lgb_6.feature_importances_)), key=itemgetter(1), reverse=True)

[('calculatedfinishedsquarefeet_2017', 30),
 ('taxamount_2016', 27),
 ('regionidzip_2017', 20),
 ('lotsizesquarefeet_2017', 19),
 ('yearbuilt_2017', 18),
 ('trans_qtr', 17),
 ('landtaxvaluedollarcnt_2016', 16),
 ('poolcnt_2017', 12),
 ('latitude_2017', 12),
 ('taxvaluedollarcnt_2016', 11),
 ('bathroomcnt_2017', 8),
 ('structuretaxvaluedollarcnt_2016', 6),
 ('propertylandusetypeid_2017', 4),
 ('taxdelinquencyflag_2016', 4),
 ('bedroomcnt_2017', 3),
 ('census_tract', 3),
 ('longitude_2017', 1),
 ('census_county', 0),
 ('census_block', 0)]

In [310]:
import pickle
pickle.dump(best_lgb_6, open('lgb_clf_6_v3.pickle', 'wb'))
# #RF_clf = pickle.load(open('RF_clf_09_29_A.pickle', 'rb'))

In [311]:
# Now fit 2017 data 

In [312]:
X_7 = df_train_2017.drop(dropcols_final, axis=1).values
y_7 = df_train_2017['logerror'].values
train_columns_7=df_train_2017.drop(dropcols_final, axis=1).columns
print(X_7.shape, y_7.shape)

(77613, 19) (77613,)


In [313]:
X_train, X_test, y_train, y_test = train_test_split(X_7, y_7, test_size=0.20)

In [314]:
lgb_train = lgb.Dataset(X_train, label=y_train, feature_name=train_columns_7, categorical_feature=cat_feat_7,free_raw_data=False )
lgb_eval = lgb.Dataset(X_test, label=y_test, feature_name=train_columns_7, categorical_feature=cat_feat_7,free_raw_data=False )

In [ ]:
# Create classifier to use.
mdl = lgb.LGBMRegressor(boosting_type= 'gbdt', 
          objective = 'regression_l1', 
          nthread = 5, 
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          subsample_freq = params['subsample_freq'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'], 
          scale_pos_weight = params['scale_pos_weight'])

# To view the default model params:
mdl.get_params().keys()

# Create the grid
grid = GridSearchCV(mdl, gridParams, verbose=1, cv=4, n_jobs=-1)
# Run the grid
grid.fit(X_train, y_train)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)
grid.grid_scores_

In [316]:
best_lgb_7=grid.best_estimator_

In [317]:
best_lgb_7

LGBMRegressor(boosting_type='gbdt', colsample_bytree=0.75, learning_rate=0.1,
       max_bin=750, max_depth=5, min_child_samples=5, min_child_weight=1,
       min_data_in_leaf=50, min_split_gain=0.5, n_estimators=8, nthread=5,
       num_leaves=18, objective='regression_l1', reg_alpha=1, reg_lambda=2,
       scale_pos_weight=1, seed=500, silent=True, subsample=0.7,
       subsample_for_bin=200, subsample_freq=1)

In [175]:
# # After further manual parameter tuning
# best_lgb_7=lgb.LGBMRegressor(boosting_type='gbdt', colsample_bytree=0.8, learning_rate=0.05,
#        max_bin=300, max_depth=8, min_child_samples=5, min_child_weight=1,
#        min_data_in_leaf=50, min_split_gain=0.5, n_estimators=50, nthread=5,
#        num_leaves=12, objective='regression_l1', reg_alpha=1, reg_lambda=2,
#        scale_pos_weight=1, seed=500, silent=True, subsample=0.75,
#        subsample_for_bin=200, subsample_freq=1, metric='mae')

In [318]:
best_lgb_7.fit(X_train, y_train)
print('Start predicting...')
# predict
y_pred = best_lgb_7.predict(X_test, num_iteration=best_lgb_7.best_iteration)
# eval
print('The mean absolute error of prediction is:', mean_absolute_error(y_test, y_pred) )

Start predicting...
The mean absolute error of prediction is: 0.0676915787245


In [319]:
sorted(list(zip(train_columns_7,best_lgb_7.feature_importances_)), key=itemgetter(1), reverse=True)

[('calculatedfinishedsquarefeet_2017', 27),
 ('latitude_2017', 16),
 ('lotsizesquarefeet_2017', 12),
 ('landtaxvaluedollarcnt_2016', 10),
 ('trans_qtr', 9),
 ('longitude_2017', 9),
 ('structuretaxvaluedollarcnt_2016', 9),
 ('taxamount_2016', 9),
 ('poolcnt_2017', 6),
 ('yearbuilt_2017', 5),
 ('regionidzip_2017', 5),
 ('bathroomcnt_2017', 3),
 ('census_block', 3),
 ('taxvaluedollarcnt_2016', 3),
 ('taxdelinquencyflag_2016', 3),
 ('bedroomcnt_2017', 1),
 ('propertylandusetypeid_2017', 1),
 ('census_tract', 1),
 ('census_county', 0)]

In [320]:
pickle.dump(best_lgb_7, open('lgb_clf_7_v3.pickle', 'wb'))
# #RF_clf = pickle.load(open('RF_clf_09_29_A.pickle', 'rb'))

Submission.....................................................................................................................

In [321]:
# Read in the submission file
sub = pd.read_csv('sample_submission.csv')

In [322]:
sub.shape

(2985217, 7)

In [323]:
sub.head(2)

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0


Prep the full property dataset for prediction

In [324]:
# Read in as needed 
#properties_combined=pd.read_csv("properties_combined_base5A")
df_pred_2016=properties_combined

In [325]:
# These were the final train columns (19)
train_columns

Index(['trans_qtr', 'calculatedfinishedsquarefeet_2017', 'bedroomcnt_2017',
       'bathroomcnt_2017', 'poolcnt_2017', 'lotsizesquarefeet_2017',
       'yearbuilt_2017', 'propertylandusetypeid_2017', 'latitude_2017',
       'longitude_2017', 'regionidzip_2017', 'census_county', 'census_tract',
       'census_block', 'landtaxvaluedollarcnt_2016',
       'structuretaxvaluedollarcnt_2016', 'taxvaluedollarcnt_2016',
       'taxdelinquencyflag_2016', 'taxamount_2016'],
      dtype='object')

In [326]:
# These were the dropped cols
dropcols_2016

['landtaxvaluedollarcnt_2017',
 'structuretaxvaluedollarcnt_2017',
 'taxvaluedollarcnt_2017',
 'taxdelinquencyflag_2017',
 'taxamount_2017',
 'rawcensustractandblock_2017',
 'censustractandblock_2017',
 'regionidneighborhood_2017',
 'regionidcity_2017',
 'regionidcounty_2017',
 'taxamount_mean_2017',
 'taxamount_mean_2016',
 'taxvaluedollarcnt_2017_mean',
 'taxvaluedollarcnt_2016_mean',
 'landtaxvaluedollarcnt_2016_mean',
 'landtaxvaluedollarcnt_2017_mean',
 'structuretaxvaluedollarcnt_2016_mean',
 'structuretaxvaluedollarcnt_2017_mean',
 'calculatedfinishedsquarefeet_2017_mean',
 'taxpersqft_2017',
 'lotsizesquarefeet_2017_mean',
 'taxperlotsizesqft_2017']

In [327]:
df_pred_2016.drop(dropcols_2016, axis=1, inplace=True)

In [328]:
df_pred_2016.shape

(2985217, 19)

In [329]:
df_pred_2016.columns

Index(['parcelid', 'calculatedfinishedsquarefeet_2017', 'bedroomcnt_2017',
       'bathroomcnt_2017', 'poolcnt_2017', 'lotsizesquarefeet_2017',
       'yearbuilt_2017', 'propertylandusetypeid_2017', 'latitude_2017',
       'longitude_2017', 'regionidzip_2017', 'census_county', 'census_tract',
       'census_block', 'landtaxvaluedollarcnt_2016',
       'structuretaxvaluedollarcnt_2016', 'taxvaluedollarcnt_2016',
       'taxdelinquencyflag_2016', 'taxamount_2016'],
      dtype='object')

In [330]:
# Add trans_qtr (4)
df_pred_2016['trans_qtr']=4

In [331]:
# Ensure columns in same order as train 
df_pred_2016=df_pred_2016[train_columns]

In [332]:
df_pred_2016.shape

(2985217, 19)

In [333]:
X_test_6 = df_pred_2016.values

In [334]:
print('Start predicting...')
y_pred_2016 = best_lgb_6.predict(X_test_6)

Start predicting...


In [335]:
len(y_pred_2016)

2985217

In [336]:
gc.collect()

214

In [337]:
prop7A=properties_combined

In [338]:
# Append predictions to dataframe
prop7A['y_pred_2016']=y_pred_2016

In [339]:
prop7A.head(2)

,parcelid,calculatedfinishedsquarefeet_2017,bedroomcnt_2017,bathroomcnt_2017,poolcnt_2017,lotsizesquarefeet_2017,yearbuilt_2017,propertylandusetypeid_2017,latitude_2017,longitude_2017,...,census_county,census_tract,census_block,landtaxvaluedollarcnt_2016,structuretaxvaluedollarcnt_2016,taxvaluedollarcnt_2016,taxdelinquencyflag_2016,taxamount_2016,trans_qtr,y_pred_2016
0,10754147,50073.210996,0.0,0.0,0,85768.0,1963.0,269,34.144442,-118.654084,...,6037,800204,1,9.0,485708.51742,9.0,0,12773.728669,4,0.024910
1,10759547,50073.210996,0.0,0.0,0,4083.0,1963.0,261,34.140430,-118.625364,...,6037,800101,1,27516.0,485708.51742,27516.0,0,12773.728669,4,0.007709


Repeat predictions for 2017 

In [340]:
# Read in as needed 
df_pred_2017=pd.read_csv("properties_combined_base5A")

In [341]:
df_pred_2017.shape

(2985217, 41)

In [342]:
# These were the final train columns (19)
train_columns

Index(['trans_qtr', 'calculatedfinishedsquarefeet_2017', 'bedroomcnt_2017',
       'bathroomcnt_2017', 'poolcnt_2017', 'lotsizesquarefeet_2017',
       'yearbuilt_2017', 'propertylandusetypeid_2017', 'latitude_2017',
       'longitude_2017', 'regionidzip_2017', 'census_county', 'census_tract',
       'census_block', 'landtaxvaluedollarcnt_2016',
       'structuretaxvaluedollarcnt_2016', 'taxvaluedollarcnt_2016',
       'taxdelinquencyflag_2016', 'taxamount_2016'],
      dtype='object')

In [343]:
# These were the dropped cols
dropcols_2017

['landtaxvaluedollarcnt_2016',
 'structuretaxvaluedollarcnt_2016',
 'taxvaluedollarcnt_2016',
 'taxdelinquencyflag_2016',
 'taxamount_2016',
 'rawcensustractandblock_2017',
 'censustractandblock_2017',
 'regionidneighborhood_2017',
 'regionidcity_2017',
 'regionidcounty_2017',
 'taxamount_mean_2017',
 'taxamount_mean_2016',
 'taxvaluedollarcnt_2017_mean',
 'taxvaluedollarcnt_2016_mean',
 'landtaxvaluedollarcnt_2016_mean',
 'landtaxvaluedollarcnt_2017_mean',
 'structuretaxvaluedollarcnt_2016_mean',
 'structuretaxvaluedollarcnt_2017_mean',
 'calculatedfinishedsquarefeet_2017_mean',
 'taxpersqft_2017',
 'lotsizesquarefeet_2017_mean',
 'taxperlotsizesqft_2017']

In [344]:
df_pred_2017.drop(dropcols_2017, axis=1, inplace=True)

In [345]:
df_pred_2017.shape

(2985217, 19)

In [346]:
df_pred_2017.columns

Index(['parcelid', 'calculatedfinishedsquarefeet_2017', 'bedroomcnt_2017',
       'bathroomcnt_2017', 'poolcnt_2017', 'lotsizesquarefeet_2017',
       'yearbuilt_2017', 'propertylandusetypeid_2017', 'latitude_2017',
       'longitude_2017', 'regionidzip_2017', 'census_county', 'census_tract',
       'census_block', 'landtaxvaluedollarcnt_2017',
       'structuretaxvaluedollarcnt_2017', 'taxvaluedollarcnt_2017',
       'taxdelinquencyflag_2017', 'taxamount_2017'],
      dtype='object')

In [347]:
# Add trans_qtr (4)
df_pred_2017['trans_qtr']=4

In [348]:
train_columns

Index(['trans_qtr', 'calculatedfinishedsquarefeet_2017', 'bedroomcnt_2017',
       'bathroomcnt_2017', 'poolcnt_2017', 'lotsizesquarefeet_2017',
       'yearbuilt_2017', 'propertylandusetypeid_2017', 'latitude_2017',
       'longitude_2017', 'regionidzip_2017', 'census_county', 'census_tract',
       'census_block', 'landtaxvaluedollarcnt_2016',
       'structuretaxvaluedollarcnt_2016', 'taxvaluedollarcnt_2016',
       'taxdelinquencyflag_2016', 'taxamount_2016'],
      dtype='object')

In [349]:
df_pred_2017.columns

Index(['parcelid', 'calculatedfinishedsquarefeet_2017', 'bedroomcnt_2017',
       'bathroomcnt_2017', 'poolcnt_2017', 'lotsizesquarefeet_2017',
       'yearbuilt_2017', 'propertylandusetypeid_2017', 'latitude_2017',
       'longitude_2017', 'regionidzip_2017', 'census_county', 'census_tract',
       'census_block', 'landtaxvaluedollarcnt_2017',
       'structuretaxvaluedollarcnt_2017', 'taxvaluedollarcnt_2017',
       'taxdelinquencyflag_2017', 'taxamount_2017', 'trans_qtr'],
      dtype='object')

In [350]:
train_columns_7=['trans_qtr', 'calculatedfinishedsquarefeet_2017', 'bedroomcnt_2017',
       'bathroomcnt_2017', 'poolcnt_2017', 'lotsizesquarefeet_2017',
       'yearbuilt_2017', 'propertylandusetypeid_2017', 'latitude_2017',
       'longitude_2017', 'regionidzip_2017', 'census_county', 'census_tract',
       'census_block', 'landtaxvaluedollarcnt_2017',
       'structuretaxvaluedollarcnt_2017', 'taxvaluedollarcnt_2017',
       'taxdelinquencyflag_2017', 'taxamount_2017']

In [351]:
# Ensure columns in same order as train 
df_pred_2017=df_pred_2017[train_columns_7]

In [352]:
df_pred_2017.shape

(2985217, 19)

In [353]:
X_test_7 = df_pred_2017.values

In [354]:
print('Start predicting...')
y_pred_2017 = best_lgb_7.predict(X_test_7)

Start predicting...


In [355]:
len(y_pred_2017)

2985217

In [356]:
gc.collect()

310

In [357]:
prop7A['y_pred_2017']=y_pred_2017

Make new df with only the parcelids and predictions 

In [358]:
prop8=prop7A[['parcelid', 'y_pred_2016', 'y_pred_2017']]

In [359]:
prop8.shape

(2985217, 3)

In [ ]:
prop8['ParcelId']=prop8['parcelid']

Now attach prediction back into sub(mission)

In [361]:
finalsub=sub.merge(prop8, how='left', on='ParcelId')

In [362]:
finalsub.head()

,ParcelId,201610,201611,201612,201710,201711,201712,parcelid,y_pred_2016,y_pred_2017
0,10754147,0,0,0,0,0,0,10754147,0.024910,0.028620
1,10759547,0,0,0,0,0,0,10759547,0.007709,0.027806
2,10843547,0,0,0,0,0,0,10843547,0.028230,0.031595
3,10859147,0,0,0,0,0,0,10859147,0.016676,0.025814
4,10879947,0,0,0,0,0,0,10879947,0.007101,0.012655


In [363]:
finalsub.drop('parcelid', axis=1, inplace=True)

In [364]:
finalsub.columns

Index(['ParcelId', '201610', '201611', '201612', '201710', '201711', '201712',
       'y_pred_2016', 'y_pred_2017'],
      dtype='object')

In [365]:
finalsub.fillna(0)

,ParcelId,201610,201611,201612,201710,201711,201712,y_pred_2016,y_pred_2017
0,10754147,0,0,0,0,0,0,0.024910,0.028620
1,10759547,0,0,0,0,0,0,0.007709,0.027806
2,10843547,0,0,0,0,0,0,0.028230,0.031595
3,10859147,0,0,0,0,0,0,0.016676,0.025814
4,10879947,0,0,0,0,0,0,0.007101,0.012655
5,10898347,0,0,0,0,0,0,0.006142,0.015294
6,10933547,0,0,0,0,0,0,0.012873,0.022680
7,10940747,0,0,0,0,0,0,0.011346,0.015661
8,10954547,0,0,0,0,0,0,0.019853,0.017549
9,10976347,0,0,0,0,0,0,0.010362,0.014657


In [366]:
finalsub['201610']=finalsub.y_pred_2016
finalsub['201611']=finalsub.y_pred_2016
finalsub['201612']=finalsub.y_pred_2016
finalsub['201710']=finalsub.y_pred_2017
finalsub['201711']=finalsub.y_pred_2017
finalsub['201712']=finalsub.y_pred_2017

In [367]:
finalsub.head()

,ParcelId,201610,201611,201612,201710,201711,201712,y_pred_2016,y_pred_2017
0,10754147,0.024910,0.024910,0.024910,0.028620,0.028620,0.028620,0.024910,0.028620
1,10759547,0.007709,0.007709,0.007709,0.027806,0.027806,0.027806,0.007709,0.027806
2,10843547,0.028230,0.028230,0.028230,0.031595,0.031595,0.031595,0.028230,0.031595
3,10859147,0.016676,0.016676,0.016676,0.025814,0.025814,0.025814,0.016676,0.025814
4,10879947,0.007101,0.007101,0.007101,0.012655,0.012655,0.012655,0.007101,0.012655


In [368]:
finalsub=finalsub[['ParcelId', '201610', '201611', '201612', '201710', '201711', '201712']]

In [369]:
finalsub.tail()

,ParcelId,201610,201611,201612,201710,201711,201712
2985212,168176230,0.013174,0.013174,0.013174,0.009705,0.009705,0.009705
2985213,14273630,0.003103,0.003103,0.003103,0.013157,0.013157,0.013157
2985214,168040630,0.001190,0.001190,0.001190,0.010521,0.010521,0.010521
2985215,168040830,0.004343,0.004343,0.004343,0.015157,0.015157,0.015157
2985216,168040430,0.003253,0.003253,0.003253,0.009446,0.009446,0.009446


In [370]:
finalsub.fillna(0, inplace=True)

In [371]:
finalsub.dtypes

ParcelId      int64
201610      float64
201611      float64
201612      float64
201710      float64
201711      float64
201712      float64
dtype: object

In [372]:
finalsub.iloc[1048578]

ParcelId    1.251849e+07
201610     -4.198778e-03
201611     -4.198778e-03
201612     -4.198778e-03
201710      7.629418e-03
201711      7.629418e-03
201712      7.629418e-03
Name: 1048578, dtype: float64

In [373]:
finalsub.shape

(2985217, 7)

In [374]:
finalsub.to_csv('Zillow_submission_v14_lgb.csv', index=False, float_format='%.4f')

Selected results from Kaggle scoring:

V3 - LGB w/o categoricals flagged 
Your submission scored 0.0658515, which is an improvement of your previous score of 0.0663010. Great job!

V4 - LGB w/categoricals flagged
Your submission scored 0.0658497, which is an improvement of your previous score of 0.0658515. Great job!

V5 - With some very basic tuning 
Your submission scored 0.0657437, which is an improvement of your previous score of 0.0658497. Great job!

V6 - RF, tuned 
Your submission scored 0.0657550, which is not an improvement of your best score. Keep trying!

V7 - RF, tuned, with transmonth added
Your submission scored 0.0658066, which is not an improvement of your best score. Keep trying!

V8 - RF, trans_qtr used instead

V9 -- Back to LGB
Your submission scored 0.0658543, which is not an improvement of your best score. Keep trying!

V10 - LGB with separate 2016/17 predictions 
Your submission scored 0.0649959, which is an improvement of your previous score of 0.0657437. Great job!

V11 - LGB, mostly same as above, but some updated 2016 property features 
Your submission scored 0.0649798, which is an improvement of your previous score of 0.0649959. Great job!

V12 - LGB, categorical features noted, boosted learning rate
Your submission scored 0.0649512, which is an improvement of your previous score of 0.0649798. Great job!

V13 - LGB, with regression_l1 parameter 
Your submission scored 0.0647780, which is an improvement of your previous score of 0.0649512. Great job!

V14 - LGB, after another roundof grid search
Your submission scored 0.0646781, which is an improvement of your previous score of 0.0647780. Great job!